In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../Data/www/styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Synopsis

Unstructured text is one of the most plentiful sources of data in many disciplines. However, because this data is unstructured (meaning that it isn't organized nicely into an excel spreadsheet) even basic analysis can be a bit more involved than with other data. In this unit we will go over the basics of textual analysis and cover:

* Techniques for **parsing** large-scale text
* Basic **bag of words** analysis
* Examining **distributions** of word usage

# Text as data

Whether it's extracting numerical data from text, or dealing with text directly, the ability to manipulate text in the form of strings is essential for any number of data science projects. More importantly, analyzing text allows for quantitative analysis in a number of areas that would be prohibitive otherwise.

As an example, think about a meeting transcript (for example, consider the meeting minutes of the [Fed](https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm)). Typical qualitative analysis would require someone to read the transcript and then summarize and infer what the main points and viewpoints of the participants were. This is an extremely important and thorough way to examine historical documents and generate insight. 


However, what if you were interested in understanding how the Fed's viewpoint on monetary policy has evolved in the last 50 years? What if you were interested in understanding which topics the Fed considered most important at different points in that same time period? Our qualitative analysis, which can be seen as a gold standard, starts to fall apart in this context because of the sheer amount of work that would be required to accomplish the task.

While a qualitative longitudinal analysis is certainly possible, you are pretty much describing 5-7 years of intensive research. Many times you simply **do not** have this amount of time and energy available. 

Quantitative textual analysis makes the task of longitudinal analysis or simply large-scale text possible. However, one important point that I want to impress upon you is that **textual analysis is not nearly as simple as it seems**. 

As an adult human, you feel an innate understanding of language and text. However, the understanding you currently have is not innate, it is the byproduct of 18+ years of learning. The computer simply **does not have 18+ years of understanding human language to fall back on**. Further, programming code to perform tasks that you innately do is one of the hardest things to accomplish, because you are frequently unaware of how many assumptions rely on and exceptions you make to understand text as a whole. In general, implicit judgments are some of the hardest to remember to put into code.

For these reasons, we're going to spend a whole day on this topic :)

# Shakespeare

Here, we're going to take a walk over to the humanities and see if we can learn anything new about our favorite author: William Shakespeare (if we all had to learn it, so should a computer, right?).

Some questions we might ask of Shakespeare's text could be...

1. Should "Othello" really be called "Iago"?

2. Can a computer learn the difference between a comedy and a tragedy?

3. Who is the most verbose Shakespearean character?

4. Who has the largest vocabulary? 

5. Did the complexity of Shakespeare's vocabulary change over time?

6. Which is Shakespeare's most feminist play?

We could think of any number of quantitative questions to pursue, each requiring slightly different skills and analytical depth. Let's start with something easy and build our way up. By the end you should have the skills to answer some of the above questions. 

# The Data

Where are we going to get data? [Project Gutenberg](http://www.gutenberg.org) has 1000's of different post-copyright books freely available in the form of easy-to-use `.txt` files. Luckily for us, this includes the complete works of William Shakespeare which we've pre-downloaded for you.

First things first, **ALWAYS LOOK AT YOUR DATA**. Now is the time to open `'../Data/Day5-Text-Analysis/Shakespeare.txt'` and get a sense for the file formatting. (You can do this quickly by clicking: [Shapespeare.txt](../Data/Day5-Text-Analysis/Shakespeare.txt)).

1. How are different plays separated from one another?
2. How is dialogue formatted?
3. What extraneous information might we want to ignore?
4. How "well behaved" is our dataset? (i.e. is the formatting general or unique for different plays?)

These are all important questions that we can only infer from qualitative visual exploration. 

# Text parsing review

Before we dive into this large file, let's do a brief refresher on some text parsing basics which will give us an excuse to spoil Hamlet for you. 

In [ ]:
spoiler_alert = ["O, I die, Horatio!\n",
    "The potent poison quite o'ercrows my spirit.\n",
    "I cannot live to hear the news from England,\n",
    "     But I do prophesy th' election lights\n",
    "On Fortinbras. He has my dying voice.\n",
    "So tell him, with th' occurrents, more and less,\n",
    "\n",             
    "Which have solicited- the rest is silence.             Dies.\n"]

If we didn't know what was in `spoiler_alert`, we would have to iterate through the lines to see what the data looks like:

In [ ]:
for line in spoiler_alert:
    print(line)

## Removing extraneous characters
Remember, `\n` is a new line character so it isn't actually displayed. It is still there, the `print` function just interprets it to say `new line` which is why we have double-spacing going on here (one line is added during the `for` loop, another is added by each `\n`).

We can get rid of the extra spaces easily enough, though:

In [ ]:
for line in spoiler_alert:
    print(line.strip('\n'))

Now the `\n` really is gone, and our text isn't double spaced. Remember, we're not actually changing `spoiler_alert`. We're only temporarily removing the `\n` as we print each line. Thus:

In [ ]:
for line in spoiler_alert:
    print(line)

The `\n` characters are still there! That's okay, I just wanted to remind you of this crucial aspect of `for` loops.

So `str.strip('\n')` removes the `\n` characters, but it still leaves a weird spacing before "But I do prophesy...". 

Instead, we could just say `str.strip()` and by default it will remove _all_ white space, which includes `\t` (tab) `\n` (new line) and '' (spaces) from both the right _and_ left ends. Note that `str.lstrip()` and `str.rstrip()` only remove characters from one end at a time.

In [ ]:
for line in spoiler_alert:
    print(line.strip())

Note that `strip()` doesn't remove anything from the middle of a line, so there is still a ton of space before 'Dies'. The `strip()` method will only remove characters from the ends of a string. This isn't a problem for us, but is important to remember.

## Getting line (index) numbers
Another handy function is `enumerate`. Suppose I wanted to know the line numbers: it's pretty trivial with this short list but we won't always work with short lists. For longer text, `enumerate` is a lifesaver. Let's see it in action:

In [ ]:
for line in enumerate(spoiler_alert):
    print(line)

`enumerate` took our list of strings named `spoiler_alert` and created a list of tuples! The **first** item in each tuple is the index within the original list (`spoiler_alert`), and the **second** item is the actual string. This will come in handy later.

## Searching in text
Let's do a little searching:

In [ ]:
for line in spoiler_alert:
    if 'The' in line:
        print(line)

Why aren't line 2 ("I cannot live...") or 7 ("Which have solicited...") printed?

**Capitalization matters!**

In [ ]:
for line in spoiler_alert:
    if 'the' in line:
        print(line)

Now we got lines 2 and 7, but we missed line 1!

Maybe we don't care about capitalization, and just want to know whether 'the' appears anywhere within the line. We'll have to standardize our search by converting each line to lowercase before searching it:

In [ ]:
for line in spoiler_alert:
    if 'the' in line.lower():
        print(line)

The same works for uppercase:

In [ ]:
for line in spoiler_alert:
    if 'the' in line.upper():
        print(line)

We didn't find anything here. Why not? 

We temporarily made the lines uppercase, so lowercase `the` never appears in any of them! We should have said:

In [ ]:
for line in spoiler_alert:
    if 'THE' in line.upper():
        print(line)

Is this the only way to find something in a line? Of course not! Programming wouldn't be fun if there weren't 1000 ways to do the same thing. 

In [ ]:
for line in spoiler_alert:
    if line.find('the') != -1:
        print(line.strip(), line.find('the'))

Not only does `find` tell us whether the text appears in the line, but also exactly where in the line the text appears (indexing from 0). If it doesn't find our search query it will return -1. These methods can also be combined in linear chains:

In [ ]:
for line in spoiler_alert:
    if line.lower().find('the') != -1: #Now we're lowercasing the line first before searching
        print(line.strip(), line.lower().find('the'))

`index` works similarly to `find`:

In [ ]:
for line in spoiler_alert:
    if line.index('the') != -1:
        print(line.strip(), line.find('the'))

Except `index` gave us a `ValueError`. Why?

When using `index`, if the string isn't found it raises an error rather than returning `-1`. We need to try something slightly different. Remember our old friend try/except?

In [ ]:
for line in spoiler_alert:
    try:
        print(line.strip(), line.lower().index('the'))
    except ValueError:
        pass

We've now learned a few different ways to search for substrings inside of strings. Capitalization really matters for all of them, and there are minor differences between methods that you need to keep in mind. 

We also learned that `enumerate` makes it easy to find the line number in which substrings occurred:

In [ ]:
for line in enumerate(spoiler_alert):
    if 'the' in line[1].lower(): #We have to look in the string, which is index one in the enumerate tuple
        print('Line number: ', line[0], '**** Line: ', line[1])#And only print the line number line[1] is the full line

We now know that 'the' appears on lines 1, 2, and 7. This might be useful at some point. For instance, what if we only want the lines between the first and last occurences of 'my'? 

**Exercise:** Use your favorite method to find lines containing the word 'my'. Next, append all lines _between_ (and including!) the lines containing the substring 'my' to their own list.

In [ ]:
lines_of_interest = []

###Place your code here



# Splitting strings

Let's do a refresher on splitting up strings. Maybe we just want a list of the words in each line rather than working with the entire line as a string:

In [ ]:
for line in spoiler_alert:
        print(line.split(','))

Here we turned each line into a list of clauses separated by commas. In lines without commas, the line became a list with a single element. In lines with commas, the string was split into separate strings based on the position of those commas (notice that the commas themselves are gone!).

We could also split our lines based on spaces to isolate individual words (kind of):

In [ ]:
for line in spoiler_alert:
        print(line.split(' '))

I said _kind of_ because while it looks like most of these are single words, it's not perfect. The last word in each line still has that pesky '`\n`', so let's combine commands to get rid of it:

In [ ]:
for line in spoiler_alert:
        print(line.strip().split(' '))

The operations were performed in order. First we stripped white space off the left and right sides of the string. Then, we split the resultant string into a list based on spaces. Now we have a list of words for each line that looks slightly better than before (we still have errors, but we'll come back to that), but what if we wanted a list of words for the entire text rather than for each line?

In [ ]:
total_list = []
for line in spoiler_alert:
    line_as_list = line.strip().split(' ')
    for word in line_as_list:
        total_list.append(word)
print(total_list)

This isn't so bad, but there are still some weird things in here that we probably don't want, like all of those empty strings that precede 'Dies' or any of the punctuation.  

In [ ]:
total_list = []
for line in spoiler_alert:
    line_as_list = line.strip().split(' ')
    for word in line_as_list:
        total_list.append(word.rstrip('!'))
print(total_list)

Here we're stripping (from the right side only) the exclamation marks. We may also want to strip the periods, colons, semi-colons, hyphens and question marks (or anything else). We can just lump all of these into `strip` and run `strip` for each word. If it doesn't find anything, it won't do anything. If it finds a question mark, it will remove it:

In [ ]:
total_list = []
for line in spoiler_alert:
    line_as_list = line.strip().split(' ')
    for word in line_as_list:
        total_list.append(word.rstrip('!?.-;:'))
print(total_list)

We will still have apostrophes, but maybe we want to keep them. What about hyphenated words? Our current practice would keep strings like 'well-known' as one long word. Perhaps we are okay with that, or perhaps not. 

Let's wrap this up:

In [ ]:
total_list = []
for line in spoiler_alert:
    line_as_list = line.strip().split(' ')
    for word in line_as_list:
        if len(word) > 0:#This will make sure the string has something in it
            total_list.append(word.rstrip('!?.,-;:').lower())
print(total_list)

Everything is lowercase so  'The' and 'the' will be recognized as the same word, and we removed both punctuation and those pesky spaces/empty strings. Finally, we can get word counts, for which we'll rely on Counter:

In [ ]:
from collections import Counter
Counter(total_list)

# Back to Shakespeare.txt

We've now created a dictionary of all the word counts in our sample text. Of course, we don't care about a tiny sample. Our real text is gigantic and contains a lot of stuff we don't need. Let's get serious and move on to some big(-ger) data. We need to start by reading in the file using python:

In [ ]:
complete_works = open('../Data/Day5-Text-Analysis/Shakespeare.txt').read()

For the next few tasks, I'm only interested  in 'Othello'. Look at where Othello begins in the text: how are we going to extract Othello and _only_ Othello from this large list of lines? How do _we_ know when Othello begins and ends?

**Exercise:** Iterate through `complete_works` and add only the lines relevant to 'Othello' to the new list `othello_lines` (hint: there might be a string that differentiates Othello from other plays. And there might be another string that signifies the end of a play. How can you use search to find those lines and get the ones you want?).

In [ ]:
othello_lines = []
###Place your code here



Before moving on, **always** make sure you really have what you _think_ you have. 

In [ ]:
print(othello_lines[0:20])
print('**************')
print(othello_lines[-20:])

# Dialogue
We should all have a line-by-line reading of Othello. Now what? Your choice here is going to depend on what you want to analyze! I'm particularly interested in how many total and _unique_ words Othello speaks. With that information, I could apply similar analyses to see which character speaks the most and who has the biggest vocabulary. 

Since we only care about Othello, it may not be worth our time to extract the character list. We'll have to do that if we want to compare vocabulary between characters, but for now let's just worry about Othello. The fact that Othello is capitalized when he speaks makes things nice and easy for us (aside: his name _probably_ isn't capialized when he is referred to in dialogue from other characters. Can we be sure? No. As a rough approximation? Sure).

```
"  CHARACTER. blahblahblah
     blahblahblah"
```

Two things jump out at me. How about you?

1. Before a character speaks there are always (usually?) *2* spaces, followed by the character in all capital letters, followed by a period. 
2. It also looks like some speeches are longer than a single line, but those speeches have 4 spaces(!). We'll make note of that for later.

In [ ]:
sample_text = othello_lines[3792:3809]
for line in sample_text:
    print(line.rstrip())

**Exercise:** Given the sample_text above, make a list of all words spoken by Othello

In [ ]:
othellos_dialogue = []

###Place your code here



In [ ]:
print(othellos_dialogue)

The code that we wrote above should (might?) be generalizable. Who knows. We'll have to check slowly.

In [ ]:
for line in othello_lines[:500]: #No reason to work with the full text while we're still learning
    if 'OTHELLO.' in line:
        print(line)
        
print('##########')

for line in othello_lines[-100:]: #No reason to work with the full text while we're still learning
    if 'OTHELLO.' in line:
        print(line)

Looks good to me. Of course, this is only finding the first line that Othello speaks, not any of his other lines. 

Let's go ahead and run our code on Othello to see how complex his vocabulary is in the first line that he speaks!

In [ ]:
othellos_dialogue = []
###Place your code here

print(Counter(othellos_dialogue))

Great! What we have done here is create what is called a **bag of words**. We've taken all of Othello's dialogue and distilled it to the individual words and their usage. 

When we use a **bag of words** approach we are typically interested in some basic statistics, such as the number of unique words used and the distribution of word occurrences. 

However, we really only have the first line. Before we start any real analysis let's get all of Othello's dialogue.

In [ ]:
###Place your code here


Now that we have all of Othello's dialogue, let's examine his word usage. What are his top 5 used words? His bottom 5?

In [ ]:
### Top and bottom words used by Othello


Interesting, right? It looks like a lot of the bottom words have some type of punctuation mark...

That's because when we do a simple split and count 'party' and 'party!' are not the same word!

Strip all the punctuation from each word and now do a count.

In [ ]:
###Your code here


Ah! much better.

What's interesting is that there seems to be a much larger set of words that have an occurence of 1 than a high occurrence. Let's examine the cumulative distribution of Othello's word usage.

In [ ]:
###Your code here


What we see in this distribution is that both the number of word occurences and the number of words with those occurrences should be logarithmically scaled. When dealing with text you will typically need to log-scale both of the axes in order to view the distribution.

Why do you think that is?

Interestingly enough, it's been shown that many natural corpuses follow [Zipf's law](https://en.wikipedia.org/wiki/Zipf%27s_law). Zipf's law basically states that the frequency of any word is inversely proportional to its rank in the frequency table. The expectation is that this will be a linear relationship when plotted on a log-log scale (which is a power-law).

Plot the rank vs. frequency chart for Othello

In [ ]:
#Your code here


**Exercise:** How does Othello's word usage compare with Iago's? Who speaks more in this play? Who has a larger vocabulary?

Rewrite your above code to be a function to return any character's dialogue. Then find Iago's dialogue and compare it to Othello's.

In [ ]:
###Place your code here



Comparing the number of unique words in a character's speech is a relatively basic measure to measure the complexity of a character's dialogue. Without paying attention to how many words total are spoken also makes it susceptible to a large imbalance in dialogue. 

One measure that researchers use to understand the complexity of text is [entropy](https://en.wikipedia.org/wiki/Entropy_%28information_theory%29). Entropy is a measure of how much disorder is in a system. The most 'ordered' text would look something like:

the the the the the the the

since it is the same word repeated over and over again. It is both highly ordered and uninformative. Given that, the more 'disordered' a character's dialogue is, the more complex the vocabulary is. 

It is calculated as:

$H(X) = -\sum p(x_i)log( p(x_i) )$

Where $x_i$ would be an individual word, $p(x_i)$ is the probability of that word appearing in the text. Since we are calculating this form data, the probability is equal to the frequency.

Write a function to compare the entropy of Iago and Othello's speech. 

In [ ]:
###Place your code here



And now we see that the entropy scores for the two characters' dialogue is nearly identical. Which means that the number of additional spoken words by Iago is having an effect on the number of unique words he uses.

**Food for thought:** Are these entropies significantly different or not? How could we test that?

**Exercise:** Our first step toward analyzing Othello was to extract out the Othello specific text from the complete works file, which we did in an arbitrary way to make our lives easier. How would you do it in an automated fashion? Specifically, read the following file and return a _list_ of _tuples_ containing the (year,name) of every play within the file. 

In [ ]:
complete_works = open('../Data/Shakespeare.txt').readlines()
play_list = []
###Place your code here


**Exercise:** How well does the code that we wrote above work on Hamlet? Romeo and Juliet? Who has the biggest vocabulary in all of Shakespeare?

In [ ]:
###Place your code here
